# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4842, uptime 0:00:33
mini_beamline                    RUNNING   pid 4843, uptime 0:00:33
random_walk                      RUNNING   pid 4858, uptime 0:00:32
random_walk_horiz                RUNNING   pid 4860, uptime 0:00:32
random_walk_vert                 RUNNING   pid 4846, uptime 0:00:33
simple                           RUNNING   pid 4847, uptime 0:00:33
thermo_sim                       RUNNING   pid 4848, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4849, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-26 20:05:51
Persistent Unique Scan ID: '7f850cf8-f7ec-43a0-b0e5-dd282572a73b'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:05:52.0 |     -1.000 |      97386 |


|         2 | 20:05:52.2 |     -0.500 |     101257 |
|         3 | 20:05:52.2 |      0.000 |     102480 |


|         4 | 20:05:52.2 |      0.500 |     101849 |
|         5 | 20:05:52.2 |      1.000 |      99951 |
+-----------+------------+------------+------------+
generator scan ['7f850cf8'] (scan num: 3)





('7f850cf8-f7ec-43a0-b0e5-dd282572a73b',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/7f850cf8-f7ec-43a0-b0e5-dd282572a73b-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-26 20:05:52
Persistent Unique Scan ID: '480f90d8-0082-456b-bbf9-0da25cf9dc4e'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:05:52.7 |          -1 | 1124206925 |


|         2 | 20:05:52.8 |          -0 | 1128004300 |


|         3 | 20:05:52.9 |           0 | 1131336839 |


|         4 | 20:05:53.1 |           0 | 1131746930 |


|         5 | 20:05:53.1 |           1 | 1130823106 |
+-----------+------------+-------------+------------+
generator scan ['480f90d8'] (scan num: 4)





('480f90d8-0082-456b-bbf9-0da25cf9dc4e',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-26 20:05:53
Persistent Unique Scan ID: '6c4d957b-cf25-4da7-a133-bd3766ebd5d1'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:05:53.7 | 1117622222 |


|         2 | 20:05:54.6 | 1059937321 |


|         3 | 20:05:55.6 | 1070826557 |


...

|        28 | 20:06:20.7 | 1126917714 |


|        29 | 20:06:21.7 | 1111193482 |


|        30 | 20:06:22.7 | 1057855231 |


+-----------+------------+------------+
generator count ['6c4d957b'] (scan num: 5)





('6c4d957b-cf25-4da7-a133-bd3766ebd5d1',)